Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Select the Python 3.6 kernel 

Note: If you get any issues with Azure AML SDK, please take a look at the [troubleshooting page](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?preserve-view=true&view=azure-ml-py#troubleshooting).

# Specify parameters
Fill in the parameters below.  If you already have IoT Hub or Azure ML workspace, then enter their information here. Otherwise, the parameter names will be used in provisioning new services.

In [2]:
# Enter the resource group in Azure where you want to provision the resources 
resource_group_name = "IoTEdgeResources"

# Enter Azure region where your services will be provisioned, for example "eastus2"
azure_region="australiacentral"

# Enter your Azure IoT Hub name 
# If you don't have an IoT Hub, pick a name to make a new one 
iot_hub_name="ProofOfConceptHub"

# Enter your IoT Edge device ID 
# If you don't have an IoT Edge device registered, pick a name to create a new one 
# This is NOT the name of your VM, but it's just an entry in your IoT Hub, so you can pick any name
iot_device_id="myEdgeDevice"

# Provide your Azure subscription ID to provision your services 
subscription_id = "LearningSpace"
# Provide your Azure ML service workspace name 
# If you don't have a workspace, pick a name to create a new one
aml_workspace_name = "myAmlWorkspace"

In [3]:
# DO NOT CHANGE THIS VALUE for this tutorial
# This is the name of the AML module you deploy to the device
module_name = "machinelearningmodule"

# Loading IOT Extension

In [5]:
# Load the IoT extension for Azure CLI
!az extension add --name azure-iot

Extension 'azure-iot' is already installed.


# Login to Azure

In [4]:
!az login --use-device-cod

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DUMRT3J6B to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "f3f0f033-eedf-4319-93b9-69372f143229",
    "id": "438a5f33-94a0-43ec-8581-18075394a5b6",
    "isDefault": true,
    "managedByTenants": [],
    "name": "LearningSpace",
    "state": "Enabled",
    "tenantId": "f3f0f033-eedf-4319-93b9-69372f143229",
    "user": {
      "name": "jan_ladiges@gmx.de",
      "type": "user"
    }
  }
]


# Setting the Azure Subscription

In [5]:
!az account set --subscription $subscription_id

# Install the Azure Machine Learning library

In [ ]:
%pip install pip -U

In [33]:
%pip install --upgrade azureml-sdk

ERROR: Could not find a version that satisfies the requirement azureml-sdk==1.39.0 (from versions: 0.1.57, 0.1.58, 0.1.59, 0.1.65, 0.1.68, 0.1.74, 0.1.80, 1.0rc83, 1.0rc85, 1.0.2, 1.0.6, 1.0.8, 1.0.10, 1.0.15, 1.0.15.1, 1.0.17, 1.0.18, 1.0.18.1, 1.0.21, 1.0.23, 1.0.30, 1.0.33, 1.0.39, 1.0.41, 1.0.43, 1.0.45, 1.0.48, 1.0.53, 1.0.55, 1.0.57, 1.0.60, 1.0.62, 1.0.65, 1.0.69, 1.0.72, 1.0.74, 1.0.76, 1.0.79, 1.0.81, 1.0.83, 1.0.85, 1.1.0rc0, 1.1.1rc0, 1.1.2rc0, 1.1.5, 1.1.5.1, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6.0, 1.7.0, 1.7.0.post1, 1.8.0, 1.9.0, 1.10.0, 1.11.0, 1.12.0, 1.13.0, 1.14.0, 1.15.0, 1.16.0, 1.17.0, 1.18.0, 1.19.0, 1.20.0, 1.21.0, 1.22.0, 1.23.0, 1.24.0, 1.25.0)
ERROR: No matching distribution found for azureml-sdk==1.39.0
Note: you may need to restart the kernel to use updated packages.


# Check core SDK version number

In [6]:
# Check core SDK version number
import azureml
from azureml.core import Workspace

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.23


# Create the Machine learning Workspace

In [ ]:
# Create ML WorkSpace 
import time
time.clock = time.time
ws = Workspace.create(subscription_id = subscription_id,
                resource_group = resource_group_name,
                name = aml_workspace_name,
                location = azure_region)

ws.write_config()

# Install the Libraries

In [ ]:
!pip install pandas
!pip install sklearn

# Load and read data set

In [7]:
# Load and read data set
import pandas
import numpy
import pickle


from sklearn import tree
from sklearn.model_selection import train_test_split

In [ ]:
temp_data = pandas.read_csv('temperature_data.csv')
temp_data

# Load features and labels

In [9]:
# Load features and labels
X, Y = temp_data[['machine_temperature', 'machine_pressure', 'ambient_temperature', 'ambient_humidity']].values, temp_data['anomaly'].values

# Train The Model

In [10]:
# Split data 65%-35% into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.35, random_state=0)

# Change regularization rate and you will likely get a different accuracy.
reg = 0.01

# Train a decision tree on the training set
#clf1 = LogisticRegression(C=1/reg).fit(X_train, Y_train)
clf1 = tree.DecisionTreeClassifier()
clf1 = clf1.fit(X_train, Y_train)
print (clf1)

DecisionTreeClassifier()


# Evaluate the accuracy

In [11]:
# Evaluate the test set
accuracy = clf1.score(X_test, Y_test)

print ("Accuracy is {}".format(accuracy))

Accuracy is 0.9907834101382489


# Serializing and testing the model

In [12]:
# Serialize the model and write to disk
f = open('model.pkl', 'wb')
pickle.dump(clf1, f)
f.close()
print ("Exported the model to model.pkl")

Exported the model to model.pkl


In [13]:
# Test the model by importing it and providing a sample data point
print("Import the model from model.pkl")
f2 = open('model.pkl', 'rb')
clf2 = pickle.load(f2)

# Normal (not an anomaly)
#X_new = [[24.90294136, 1.44463889, 20.89537849, 24]]
#X_new = [[33.40859853, 2.413637808, 20.89162813, 26]]
#X_new = [[34.42109181, 2.528985143, 21.23903786, 25]]

# Anomaly
X_new = [[33.66995566, 2.44341267, 21.39450979, 26]]
#X_new = [[105.5457931, 10.63179922, 20.62029994, 26]]

print ('New sample: {}'.format(X_new))

pred = clf2.predict(X_new)
print('Predicted class is {}'.format(pred))

Import the model from model.pkl
New sample: [[33.66995566, 2.44341267, 21.39450979, 26]]
Predicted class is [1]


# Register Model to cloud

In [ ]:
%pip install azure.cli.core --upgrade

In [18]:
#get workspace
import time
from azureml.core.authentication import AzureCliAuthentication 
cli_auth = AzureCliAuthentication()
from azureml.core.authentication import MsiAuthentication
msi_auth = MsiAuthentication()
time.clock=time.time
ws = Workspace.get(
    name = aml_workspace_name,
    subscription_id= subscription_id,
    resource_group= resource_group_name,
    auth=cli_auth
)

TypeError: __init__() got an unexpected keyword argument 'async_persist'

In [24]:
#Register Model to cloud
from azureml.core.model import Model

model = Model.register(model_path = "model.pkl",
                       model_name = "model.pkl",
                       tags = {'area': "anomaly", 'type': "classification"},
                       description = "Sample anomaly detection model for IOT tutorial",
                       workspace = ws)

NameError: name 'ws' is not defined

In [ ]:
#print model 
print(model.name, model.description, model.version, sep = '\t')

# Write out python file iot_score.py

In [ ]:
%%writefile iot_score.py
# This script generates the scoring file
# with the init and run functions needed to 
# operationalize the anomaly detection sample

import pickle
import json
import pandas
import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(input_str):
    try:
        input_json = json.loads(input_str)
        input_df = pandas.DataFrame([[input_json['machine']['temperature'],input_json['machine']['pressure'],input_json['ambient']['temperature'],input_json['ambient']['humidity']]])
        pred = model.predict(input_df)
        print("Prediction is ", pred[0])
    except Exception as e:
        result = str(e)
        
    if pred[0] == 1:
        input_json['anomaly']=True
    else:
        input_json['anomaly']=False
        
    return [json.dumps(input_json)]


# Loading Azure ML workspace

In [ ]:
# Initialize a workspace object from persisted configuration
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Create docker Image

In [ ]:
# Create docker Image This specifies the dependencies to include in the environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(
    conda_packages=['pandas', 'scikit-learn', 'numpy', 'pip=20.1.1'],
    pip_packages=['azureml-sdk'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="iot_score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "iot", 'type': "classification"},
                                 description = "IOT Edge anomaly detection demo")


image = Image.create(name = "tempanomalydetection",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

In [ ]:
image.wait_for_creation(show_output = True)

In [ ]:
for i in Image.list(workspace = ws,tags = ["area"]):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

# Testing the model on Azure container Instance

In [ ]:
#test Model on Azure Container

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "iot", 'type': "classification"}, 
                                               description = 'IOT Edge anomaly detection demo')

In [ ]:
from azureml.core.webservice import AciWebservice

aci_service_name = 'tempsensor-iotedge-ml-1'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)

# Testing web service

In [ ]:
#testing Web Service 

import json

# Anomaly
#test_sample = json.dumps({ "machine": { "temperature": 33.66995566, "pressure": 2.44341267 }, \
#                          "ambient": { "temperature": 21.39450979, "humidity": 26 },\
#                          "timeCreated": "2017-10-27T18:14:02.4911177Z" })

# Normal
test_sample = json.dumps({ "machine": { "temperature": 31.16469009, "pressure": 2.158002669 }, \
                          "ambient": { "temperature": 21.17794693, "humidity": 25 },\
                          "timeCreated": "2017-10-27T18:14:02.4911177Z" })

test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aci_service.run(input_data = test_sample)
print(prediction)

# Getting container details

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

# Deploying container to the Edge device

In [ ]:
#Deploying Container to Edge 

file = open('iot-workshop-deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json